In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()

In [ ]:

from langchain.agents import tool

In [ ]:
import requests
@tool
def  open_fda_label_tool(drug_name:str, limit :int =1) -> str:
    """
    Fetches safety label information for a given drug from OpenFDA.

    Args:
        drug_name (str): Brand or generic name of the drug.
        limit (int): Number of records to retrieve (max 100).

    Returns:
        str: A formatted string with drug safety details.
    """
    BASE_URL = "https://api.fda.gov/drug/label.json"

    if limit >100:
        limit=100
    
    params={
         "search": f'openfda.brand_name:"{drug_name}"',
        "limit": 1
            }
    
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if "results" not in data:
            return f"No FDA label data found for {drug_name}."

        output = []
        for result in data["results"]:
            brand = result.get("openfda", {}).get("brand_name", ["N/A"])[0]
            manufacturer = result.get("openfda", {}).get("manufacturer_name", ["N/A"])[0]
            warnings = "\n".join(result.get("warnings", ["No warnings found"]))
            boxed_warning = "\n".join(result.get("boxed_warning", ["No boxed warning found"]))
            adverse = "\n".join(result.get("adverse_reactions", ["No adverse reactions listed"]))

            output.append(
                f"Brand: {brand}\n"
                f"Manufacturer: {manufacturer}\n\n"
                f"⚠ Warnings:\n{warnings}\n\n"
                f"🚨 Boxed Warning:\n{boxed_warning}\n\n"
                f"💊 Adverse Reactions:\n{adverse}\n"
            )

        return "\n" + ("-"*50 + "\n").join(output)

    except requests.exceptions.RequestException as e:
        return f"Error querying OpenFDA API: {e}"




In [ ]:
import requests

BASE_URL = "https://api.fda.gov/drug/enforcement.json"
@tool
def openfda_recalls_tool(drug_name: str, limit: int = 5) -> str:
    """
    Fetches recent FDA recall/enforcement reports for a given drug.

    Args:
        drug_name (str): Brand or generic drug name.
        limit (int): Number of recall records to retrieve (max 100).

    Returns:
        str: A formatted string summarizing recall details.
    """
    params = {
        "search": f'product_description:"{drug_name}"',
        "limit": limit
    }

    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if "results" not in data:
            return f"No FDA recall data found for {drug_name}."

        output = []
        for recall in data["results"]:
            recall_number = recall.get("recall_number", "N/A")
            status = recall.get("status", "N/A")
            classification = recall.get("classification", "N/A")
            reason = recall.get("reason_for_recall", "No reason provided.")
            date = recall.get("report_date", "N/A")
            recalling_firm = recall.get("recalling_firm", "N/A")
            product_desc = recall.get("product_description", "N/A")

            output.append(
                f"📅 Date: {date}\n"
                f"🏢 Firm: {recalling_firm}\n"
                f"💊 Product: {product_desc}\n"
                f"⚠ Classification: {classification}\n"
                f"📝 Reason: {reason}\n"
                f"📌 Status: {status}\n"
                f"🔢 Recall #: {recall_number}\n"
            )

        return "\n" + ("-"*50 + "\n").join(output)

    except requests.exceptions.RequestException as e:
        return f"Error querying OpenFDA Recall API: {e}"


In [ ]:
from langchain.agents import AgentType,initialize_agent, Tool
tools=[Tool(name="open_fda_label_tool",description=" Tool to get fda label info",func=open_fda_label_tool),
       Tool(name="openfda_recalls_tool",description=" Tool to get fda RECALL info",func=openfda_recalls_tool)   ,
       Tool(name="Intermediate Answer",description="useful for when you need to ask with search",func=search.run)
    ]


In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name='gemma2-9b-it')

In [ ]:
agent=initialize_agent(tools,llm,verbose=True,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handle_parsing_errors=True)

In [ ]:
result1 = agent.run("Tell me the label info for Tremfya and  recall info for its top competitor by market share.")
print(result1)